In [3]:
import cv2
import numpy as np
from pathlib import Path

from hfnet.settings import EXPER_PATH
from notebooks.utils import plot_images, plot_matches, add_frame

import tensorflow as tf
print(tf.__version__)
from tensorflow.python.saved_model import tag_constants
tf.contrib.resampler  # import C++ op
from os import listdir
from os.path import isfile, join
import os, sys


%load_ext autoreload
%autoreload 2
%matplotlib inline

1.12.0


# Create HF-Net model for inference

In [4]:
class HFNet:
    def __init__(self, model_path, outputs):
        self.session = tf.Session()
        self.image_ph = tf.placeholder(tf.float32, shape=(None, None, 3))

        net_input = tf.image.rgb_to_grayscale(self.image_ph[None])
        tf.saved_model.loader.load(
            self.session, [tag_constants.SERVING], str(model_path),
            clear_devices=True,
            input_map={'image:0': net_input})

        graph = tf.get_default_graph()
        self.outputs = {n: graph.get_tensor_by_name(n+':0')[0] for n in outputs}
        self.nms_radius_op = graph.get_tensor_by_name('pred/simple_nms/radius:0')
        self.num_keypoints_op = graph.get_tensor_by_name('pred/top_k_keypoints/k:0')
        
    def inference(self, image, nms_radius=4, num_keypoints=1000):
        inputs = {
            self.image_ph: image[..., ::-1].astype(np.float),
            self.nms_radius_op: nms_radius,
            self.num_keypoints_op: num_keypoints,
        }
        return self.session.run(self.outputs, feed_dict=inputs)

model_path = Path(EXPER_PATH, 'saved_models/hfnet')
outputs = ['global_descriptor', 'keypoints', 'local_descriptors']
hfnet = HFNet(model_path, outputs)

[10/23/2019 11:14:12 INFO] Restoring parameters from /home/pang/software/hfnet_venv/hfnet_exp/saved_models/hfnet/variables/variables


# Compute global descriptors and local features for query and database

In [ ]:
# inference images
dataset_folder = "/home/pang/data/dataset/iros2019slam/office/office-1-1/"
image_list_folder = dataset_folder + "fisheye1"
kp_record_folder = dataset_folder + "fisheye1_hfnet_kps"
local_des_record_folder = dataset_folder + "fisheye1_hfnet_local_des"
global_des_record_folder = dataset_folder + "fisheye1_hfnet_global_des"
if not os.path.exists(kp_record_folder):
    os.mkdir(kp_record_folder)
if not os.path.exists(local_des_record_folder):
    os.mkdir(local_des_record_folder)
if not os.path.exists(global_des_record_folder):
    os.mkdir(global_des_record_folder)


# query = hfnet.inference(image_query)
# # print (query)
# # print ((query['global_descriptor']))
# print ((query['global_descriptor']).shape)
# print ((query['keypoints']).shape)
# print (len(query['keypoints']))
# print ((query['local_descriptors']).shape)
# print (len(query['local_descriptors']))

image_files = [f[:-4] for f in listdir(image_list_folder) if isfile(join(image_list_folder, f))]
image_files.sort(key=lambda f: int(f))

total = len(image_files)
cnt = 0
# print (image_files)
for image in image_files:
    image_name = image_list_folder + "/" + image + ".png";
    print (image_name)
    query_image = cv2.imread(image_name)[:, :, ::-1]
    print (query_image.shape)
    kp_file = kp_record_folder + "/" + image + ".txt"
    local_des_file = local_des_record_folder + "/" + image + ".txt"
    global_des_file = global_des_record_folder + "/" + image + ".txt"
    
    kp_ofs = open(kp_file, 'w')
    local_des_ofs = open(local_des_file, 'w')
    global_des_ofs = open(global_des_file, 'w')
    
    # inference 
    query_info = hfnet.inference(query_image)
    
    # write kp
    for kp in query_info['keypoints']:
        kp_ofs.write(str(kp[0]) + " " + str(kp[1]) + "\n")
#         print (type(kp))
    kp_ofs.close()
        
        
    # write global des
    for global_des in query_info['global_descriptor']:
        global_des_ofs.write(str(global_des) + "\n")
    
    global_des_ofs.close()
    
            
    # write local des
    for local_des in query_info['local_descriptors']:
        for i in local_des:
            local_des_ofs.write(str(i) + " ")
        local_des_ofs.write("\n")
    local_des_ofs.close()
    
    cnt  = cnt + 1
    process = str(cnt) + " / " + str(total)
    print (process)
        


/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000083949067.png
(800, 848, 3)
1 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000083982400.png
(800, 848, 3)
2 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000084015734.png
(800, 848, 3)
3 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000084049067.png
(800, 848, 3)
4 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000084082392.png
(800, 848, 3)
5 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000084115725.png
(800, 848, 3)
6 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000084149059.png
(800, 848, 3)
7 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000084182392.png
(800, 848, 3)
8 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000084215726.png
(800, 848, 3)
9 / 810
/home/pang/data/dataset/iros2019slam/

77 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000086515654.png
(800, 848, 3)
78 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000086548988.png
(800, 848, 3)
79 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000086582276.png
(800, 848, 3)
80 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000086615609.png
(800, 848, 3)
81 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000086648943.png
(800, 848, 3)
82 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000086682276.png
(800, 848, 3)
83 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000086715609.png
(800, 848, 3)
84 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000086748943.png
(800, 848, 3)
85 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000086782276.png
(800, 848, 3)
86 / 810
/home/pang/data/dat

153 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000089048908.png
(800, 848, 3)
154 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000089082175.png
(800, 848, 3)
155 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000089115508.png
(800, 848, 3)
156 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000089148842.png
(800, 848, 3)
157 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000089182175.png
(800, 848, 3)
158 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000089215508.png
(800, 848, 3)
159 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000089248842.png
(800, 848, 3)
160 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000089282175.png
(800, 848, 3)
161 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000089315509.png
(800, 848, 3)
162 / 810
/home/pan

229 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000091582087.png
(800, 848, 3)
230 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000091615421.png
(800, 848, 3)
231 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000091648754.png
(800, 848, 3)
232 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000091682087.png
(800, 848, 3)
233 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000091715421.png
(800, 848, 3)
234 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000091748754.png
(800, 848, 3)
235 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000091782088.png
(800, 848, 3)
236 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000091815421.png
(800, 848, 3)
237 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000091848754.png
(800, 848, 3)
238 / 810
/home/pan

305 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000094115324.png
(800, 848, 3)
306 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000094148658.png
(800, 848, 3)
307 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000094181991.png
(800, 848, 3)
308 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000094215324.png
(800, 848, 3)
309 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000094248658.png
(800, 848, 3)
310 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000094281991.png
(800, 848, 3)
311 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000094315325.png
(800, 848, 3)
312 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000094348658.png
(800, 848, 3)
313 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000094381991.png
(800, 848, 3)
314 / 810
/home/pan

381 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000096648526.png
(800, 848, 3)
382 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000096681859.png
(800, 848, 3)
383 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000096715192.png
(800, 848, 3)
384 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000096748526.png
(800, 848, 3)
385 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000096781859.png
(800, 848, 3)
386 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000096815193.png
(800, 848, 3)
387 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000096848526.png
(800, 848, 3)
388 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000096881859.png
(800, 848, 3)
389 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000096915193.png
(800, 848, 3)
390 / 810
/home/pan

457 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000099181755.png
(800, 848, 3)
458 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000099215088.png
(800, 848, 3)
459 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000099248422.png
(800, 848, 3)
460 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000099281755.png
(800, 848, 3)
461 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000099315089.png
(800, 848, 3)
462 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000099348422.png
(800, 848, 3)
463 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000099381755.png
(800, 848, 3)
464 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000099415089.png
(800, 848, 3)
465 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000099448422.png
(800, 848, 3)
466 / 810
/home/pan

533 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000101714983.png
(800, 848, 3)
534 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000101748317.png
(800, 848, 3)
535 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000101781650.png
(800, 848, 3)
536 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000101814983.png
(800, 848, 3)
537 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000101848317.png
(800, 848, 3)
538 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000101881650.png
(800, 848, 3)
539 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000101914984.png
(800, 848, 3)
540 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000101948317.png
(800, 848, 3)
541 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000101981650.png
(800, 848, 3)
542 / 810
/home/pan

609 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000104248244.png
(800, 848, 3)
610 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000104281578.png
(800, 848, 3)
611 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000104314911.png
(800, 848, 3)
612 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000104348244.png
(800, 848, 3)
613 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000104381578.png
(800, 848, 3)
614 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000104414911.png
(800, 848, 3)
615 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000104448245.png
(800, 848, 3)
616 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000104481578.png
(800, 848, 3)
617 / 810
/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/1560000104514911.png
(800, 848, 3)
618 / 810
/home/pan